In [2075]:
import pandas as pd 
import numpy as np 

In [2076]:
students = "../raw_data/students_complete.csv"
schools = "../raw_data/schools_complete.csv"
students_df = pd.read_csv(students)
schools_df = pd.read_csv(schools)

In [2077]:
students_df.head()

,Student ID,name,gender,grade,school,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [2078]:
schools_df.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [2079]:
#Create variables for key school metrics

totalSchools = schools_df['name'].count()
totalStudents = students_df['name'].count()
totalBudget = schools_df['budget'].sum()
avgMath = students_df['math_score'].mean()
avgReading = students_df['reading_score'].mean()

In [2080]:
#Identify count of passing reading and math scores then calculate percent passing
passMath_df = students_df.loc[students_df['math_score'] > 69,['math_score']]
passMathCount = passMath_df['math_score'].count()

passReading_df = students_df.loc[students_df['reading_score'] > 69,['reading_score']]
passReadingCount = passReading_df['reading_score'].count()

passMath = (passMathCount/totalStudents) 
passReading = (passReadingCount/totalStudents)

overallPass = ((passMath + passReading)/2)

In [2081]:
#Insert key school metrics variables into a dictionary to store in a summary dataframe

data = {'Total Schools':[totalSchools],
        'Total Students':[totalStudents],
        'Total Budget':[totalBudget],
        'Average Math Score':[avgMath],
        'Average Reading Score':[avgReading],
        '% Passing Math':[passMath],
        '% Passing Reading':[passReading],
        'Average of Math and Reading Pass':[overallPass]
         }
summary_pd = pd.DataFrame(data, columns = ['Total Schools', 'Total Students','Total Budget','Average Math Score','Average Reading Score', '% Passing Math','% Passing Reading','Average of Math and Reading Pass'])
summary_pd['Total Budget'] = summary_pd['Total Budget'].map("${:,.2f}".format)
summary_pd['% Passing Math'] = summary_pd['% Passing Math'].map("{:,.2%}".format)
summary_pd['% Passing Reading'] = summary_pd['% Passing Reading'].map("{:,.2%}".format)
summary_pd['Average of Math and Reading Pass'] = summary_pd['Average of Math and Reading Pass'].map("{:,.2%}".format)

#PART ONE HW SOLUTION, summary of all schools 
summary_pd.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Average of Math and Reading Pass
0,15,39170,"$24,649,428.00",78.985371,81.87784,74.98%,85.81%,80.39%


In [2082]:
#Group all students by school and score means

scores_df = students_df.groupby(['school']).mean()
scores_df = scores_df[['reading_score', 'math_score']]

In [2083]:
#Convert both dataframes to string so that they can be merged 

scores_df = scores_df.astype(str)
schools_df = schools_df.astype(str)

In [2084]:
#reset indexes and sort by school so they can be joined accurately 

scores_df = scores_df.reset_index()
schools_df = schools_df.sort_values('name')
schools_df = schools_df.reset_index()

In [2085]:
#Create a joined df for totaled key values

schoolTotals = schools_df.join(scores_df)
schoolTotals = schoolTotals[['name', 'type', 'size', 'budget', 'reading_score', 'math_score']]
schoolTotals.rename(columns={"name": "school"}, inplace=True)

In [2086]:
#find students who passed to get a pass rate by school 

reading = students_df[students_df['reading_score'] > 69].groupby('school')['reading_score'].count()
math = students_df[students_df['math_score'] > 69].groupby('school')['math_score'].count()

In [2087]:
#join math and reading pass rates and convert to string to join 

df = reading.to_frame().join(math)
df.astype(str)
df.rename(columns={"reading_score": "reading_pass_count", 'math_score': 'math_pass_count'}, inplace=True)
schoolTotals= schoolTotals.join(df, on='school')

In [2088]:
#convert all numeric values back to float so calculations can be done 

schoolTotals['size'] = schoolTotals['size'].astype(float)
schoolTotals['budget'] = schoolTotals['budget'].astype(float)
schoolTotals['reading_score'] = schoolTotals['reading_score'].astype(float)
schoolTotals['math_score'] = schoolTotals['math_score'].astype(float)

In [2089]:
#create column with metrics by school

schoolTotals['Per Student Budget']=schoolTotals['budget']/schoolTotals['size']
schoolTotals['% Passing Reading']=schoolTotals['reading_pass_count']/schoolTotals['size']
schoolTotals['% Passing Math']=schoolTotals['math_pass_count']/schoolTotals['size']
schoolTotals['Overall Passing Rate']=((schoolTotals['% Passing Math'] + schoolTotals['% Passing Reading'])/2)


In [2090]:
#PART TWO HW SOLUTION, key metrics listed by each school 

schoolTotals = schoolTotals[['school','type','size','budget','Per Student Budget', 'reading_score',
                           'math_score','% Passing Reading','% Passing Math', 'Overall Passing Rate']]
schoolTotals

,school,type,size,budget,Per Student Budget,reading_score,math_score,% Passing Reading,% Passing Math,Overall Passing Rate
0,Bailey High School,District,4976.0,3124928.0,628.0,81.033963,77.048432,0.819333,0.666801,0.743067
1,Cabrera High School,Charter,1858.0,1081356.0,582.0,83.975780,83.061895,0.970398,0.941335,0.955867
2,Figueroa High School,District,2949.0,1884411.0,639.0,81.158020,76.711767,0.807392,0.659885,0.733639
3,Ford High School,District,2739.0,1763916.0,644.0,80.746258,77.102592,0.792990,0.683096,0.738043
4,Griffin High School,Charter,1468.0,917500.0,625.0,83.816757,83.351499,0.971390,0.933924,0.952657
5,Hernandez High School,District,4635.0,3022020.0,652.0,80.934412,77.289752,0.808630,0.667530,0.738080
6,Holden High School,Charter,427.0,248087.0,581.0,83.814988,83.803279,0.962529,0.925059,0.943794
7,Huang High School,District,2917.0,1910635.0,655.0,81.182722,76.629414,0.813164,0.656839,0.735002
8,Johnson High School,District,4761.0,3094650.0,650.0,80.966394,77.072464,0.812224,0.660576,0.736400
9,Pena High School,Charter,962.0,585858.0,609.0,84.044699,83.839917,0.959459,0.945946,0.952703


In [2091]:
#HW SOLUTION PART 3, top 4 performing schools by pass rank 

passRank['% Passing Math'] = passRank['% Passing Math'].map("{:,.2}%".format)  #FIX HERE
passRank['% Passing Reading'] = passRank['% Passing Reading'].map("{:,.2}%".format)
passRank['Overall Passing Rate'] = passRank['Overall Passing Rate'].map("{:,.2}%".format)

passRank = schoolTotals.sort_values(['Overall Passing Rate'], ascending=False)
passRank.head()

,school,type,size,budget,Per Student Budget,reading_score,math_score,% Passing Reading,% Passing Math,Overall Passing Rate
1,Cabrera High School,Charter,1858.0,1081356.0,582.0,83.975780,83.061895,0.970398,0.941335,0.955867
12,Thomas High School,Charter,1635.0,1043130.0,638.0,83.848930,83.418349,0.973089,0.932722,0.952905
9,Pena High School,Charter,962.0,585858.0,609.0,84.044699,83.839917,0.959459,0.945946,0.952703
4,Griffin High School,Charter,1468.0,917500.0,625.0,83.816757,83.351499,0.971390,0.933924,0.952657
13,Wilson High School,Charter,2283.0,1319574.0,578.0,83.989488,83.274201,0.965396,0.938677,0.952037


In [2092]:
#HW SOLUTION PART 4, bottom 4 performining schools by pass rank

passRank.tail()

,school,type,size,budget,Per Student Budget,reading_score,math_score,% Passing Reading,% Passing Math,Overall Passing Rate
3,Ford High School,District,2739.0,1763916.0,644.0,80.746258,77.102592,0.792990,0.683096,0.738043
8,Johnson High School,District,4761.0,3094650.0,650.0,80.966394,77.072464,0.812224,0.660576,0.736400
7,Huang High School,District,2917.0,1910635.0,655.0,81.182722,76.629414,0.813164,0.656839,0.735002
2,Figueroa High School,District,2949.0,1884411.0,639.0,81.158020,76.711767,0.807392,0.659885,0.733639
10,Rodriguez High School,District,3999.0,2547363.0,637.0,80.744686,76.842711,0.802201,0.663666,0.732933


In [2093]:
#Group all students in original students csv by their grade and average scores

gradeAverages = students_df.groupby(['grade']).mean()
gradeAverages = gradeAverages.reset_index()


In [2094]:
#HW SOLUTION PART 5, English averages by grade

readingAverages = gradeAverages[['grade','reading_score']]
readingAverages

,grade,reading_score
0,10th,81.874410
1,11th,81.885714
2,12th,81.819851
3,9th,81.914358


In [2095]:
#HW SOLUTION PART 6, math averages by grade

mathAverages = gradeAverages[['grade','math_score']]
mathAverages

,grade,math_score
0,10th,78.941483
1,11th,79.083548
2,12th,78.993164
3,9th,78.935659


In [2096]:
#use schoolTotals to group by Charter or District

typeScores = passRank.groupby(['type']).mean()

In [2097]:
#HW SOLUTION PART 9, scores by Charter or District school type

typeScores = typeScores[['reading_score', 'math_score', '% Passing Reading', '% Passing Math', 'Overall Passing Rate']]
typeScores

,reading_score,math_score,% Passing Reading,% Passing Math,Overall Passing Rate
type,,,,,
Charter,83.896421,83.473852,0.965865,0.936208,0.951037
District,80.966636,76.956733,0.807991,0.665485,0.736738


In [2098]:
#school size labels and bins
bins = [0, 1000, 2000, 5000]
sizes = ['small','medium', 'large']

#Create column for sorting and groupby object
pd.cut(schoolTotals["size"], bins, labels=sizes)
schoolTotals['School Sizes'] = pd.cut(schoolTotals["size"], bins, labels=sizes)
schoolSizes = schoolTotals.groupby('School Sizes')

In [2099]:
#HW PART 8 SOLUTION, key metrics by school sizes

schoolSizes = schoolSizes[['reading_score','math_score','% Passing Reading','% Passing Math','Overall Passing Rate']].mean()
schoolSizes

,reading_score,math_score,% Passing Reading,% Passing Math,Overall Passing Rate
School Sizes,,,,,
small,83.929843,83.821598,0.960994,0.935502,0.948248
medium,83.864438,83.374684,0.967907,0.935997,0.951952
large,81.344493,77.746417,0.827666,0.699634,0.763650


In [2100]:
#school funding bins and labels
bins = [0, 585, 615, 645, 700]
amounts = ['<$585', '585-$614', '615-$644', '645-$699']

#Create column for sorting and groupy object
pd.cut(schoolTotals["Per Student Budget"], binss, labels=amounts)
schoolTotals['Budget Per Student'] = pd.cut(schoolTotals["Per Student Budget"], bins, labels=amounts)
studentBudget = schoolTotals.groupby('Budget Per Student')

In [2101]:
#HW ANSWER PART 8, key metrics by per student funding 

studentBudget = studentBudget[['reading_score','math_score','% Passing Reading','% Passing Math','Overall Passing Rate']].mean()
studentBudget

,reading_score,math_score,% Passing Reading,% Passing Math,Overall Passing Rate
Budget Per Student,,,,,
<$585,83.933814,83.455399,0.966109,0.934601,0.950355
585-$614,83.885211,83.599686,0.959003,0.942309,0.950656
615-$644,81.891436,79.079225,0.861066,0.756682,0.808874
645-$699,81.027843,76.997210,0.811340,0.661648,0.736494
